In [23]:
####
# @author: Nwamaka Nzeocha and Fabian Okeke
# @course: CS 5304/Data Science in the Wild
####

# Environment setup

 - useful tensor flow notebook: 
      - http://bit.ly/1NjhcfQ
      - http://katbailey.github.io/post/neural-nets-in-python/
 - pip install scikit-learn
 - pip install --upgrade https://storage.googleapis.com/tensorflow/mac/tensorflow-0.8.0-py2-none-any.whl
 - dataset:
     - https://s3.amazonaws.com/giyengar-cs5304/a4_smvl_tst.gz
     - https://s3.amazonaws.com/giyengar-cs5304/a4_smvl_val.gz
     - https://s3.amazonaws.com/giyengar-cs5304/a4_smvl_trn.gz

# Add libraries and relevant functions

In [192]:
import random
import os.path
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import load_svmlight_file


#############
# helper functions
#############
def exists(filepath):
    return os.path.exists(filepath)

def get_data(filepath):
    data = load_svmlight_file(filepath)
    return data[0], data[1]

# Load and train model

In [173]:
# get all training files
cwd = os.getcwd()
tr_files = [f for f in os.listdir(cwd) if 'training' in f]

# create model
lr = SGDClassifier(loss="log")
classes = [1, -1]

start = time()
for f in tr_files:
    X_train, y_train = get_data(f)
    lr.partial_fit(X_train, y_train, classes=classes)
    print '%s finished...' % f
print 'All completed.'
print 'Spent time:', time() - star

trainingaa
file size: 891115827
trainingaa finished...
trainingab
file size: 947808879
trainingab finished...
trainingac
file size: 900102567
trainingac finished...
trainingad
file size: 892317123
trainingad finished...
trainingae
file size: 798194430
trainingae finished...
All completed.


In [187]:
lr

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

# Find best model parameters using validation set

# train & validate with smaller data

In [202]:
from time import time

# training
start = time()
X_train, y_train = get_data('trainSmall')
print X_train.shape
print "Training time: ", time() - start

# validation
start = time()
X_va, y_va = get_data('valSmall')
print "Validation time: ", time() - start

# prediction accuracy
prediction = lr.predict(X_va)
print 'Validation accuracy:', metrics.accuracy_score(y_va, prediction)

(10000, 18898)
Training time:  1.97808504105
Validation time:  1.82869791985
Validation accuracy: 0.9015


# Fit logistic regression model

In [205]:
model_LR = LogisticRegression()
model_LR.fit(X_train, y_train)
predicted_LR = model_LR.predict(X_va)
print 'model_LR accuracy:', metrics.accuracy_score(y_va, predicted_LR)

model_LR accuracy: 0.9108


# Using tensor flow

In [209]:
from sklearn.neural_network import MLPClassifier

model_MLP = MLPClassifier(algorithm='l-bfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
model_MLP.fit(X_train, y_train)
predicted_MLP = model_MLP.predict(X_va)

model_MLP

ImportError: cannot import name MLPClassifier

# Pre calibration Brier loss and Brier curves

In [ ]:
print "pre-calibration brier loss:"
print "pre-calibration brier curve:"

# Post calibration Brier loss and Brier curves

In [ ]:
print "post-calibration brier loss:"
print "post-calibration brier curve:"

# ROC curves, Accuracy scores, AUC scores

In [ ]:
y_predictions = {
    'Logistic Regression': predicted_LR,
    'Tensor Flow': predicted_MLP,
}

fig, axes = plt.subplots(1, len(y_predictions), sharey=True)
fig.set_size_inches(15,5)
i = 0

for title, prediction in y_predictions.items():
    accuracy = metrics.accuracy_score(y_va, prediction)
    fulltitle = title + ' (accuracy = %0.2f)' % accuracy
    fpr, tpr, thresholds = metrics.roc_curve(y_va, prediction, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    label = 'AUC = %0.2f' % auc

    axes[i].plot(fpr, tpr, label=label)
    axes[i].set_title(fulltitle)
    axes[i].legend(loc='lower right')
    i += 1